In [6]:
%load_ext autoreload 
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
from src.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
m = Chain([
    BayesianConv2d(1, 16, 3, padding=1),
])
out = m(torch.ones(1, 1, 28, 28)).rsample()
loss = F.mse_loss(out, torch.zeros_like(out))
loss.backward()


In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.ToTensor()
data_train = datasets.MNIST(root="", train=True, transform=transform)
data_test = datasets.MNIST(root="", train=False, transform=transform)
loader_train = DataLoader(data_train, batch_size=1, shuffle=True)



torch.Size([20, 10])